# {{cookiecutter.notebook_name}} - life

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pathlib

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch, helpers, plotutils
import holoviews as hv
from holoviews import opts

In [ ]:
%matplotlib inline
hv.extension('bokeh')
pd.set_option('display.max_columns', 70)

In [ ]:
cellpy_batch_file = "cellpy_batch_" + {{cookiecutter.session_id}} + ".json"

In [ ]:
b = batch.init(default_log_level="INFO", db_reader=None)
b.experiment.journal.from_file(cellpy_batch_file)
b.link()
cell_labels = b.experiment.cell_names
print(" cell labels: ".center(80, "-"))
print(" - ".join(cell_labels))

## Splitting

In [ ]:
#-------creating-summaries------
first_lifetime_cycle = 23
norm_cycles = [2, 3, 4, 5]
life_summary_frames = []
rate_summary_frames =[]
keys = []

for cell_id in b.experiment.cell_names:
    # print(f"Processing [{cell_id}]", end=" ")
    keys.append(cell_id)  # replace this with another key-name (e.g. cell number)
    
    cell = b.experiment.data[cell_id]
     
    # needed now due to changes in cellpy after original files were processed:
    cell.make_step_table() 
    cell.make_summary()
    
    # cell = helpers.add_c_rate(cell)
    # cell = helpers.add_normalized_cycle_index(cell)
    cell = helpers.add_normalized_capacity(cell, norm_cycles)
    cell = helpers.add_areal_capacity(cell, cell_id, b.experiment.journal)
    
    cell_rate, cell_life = helpers.split_experiment(cell, first_lifetime_cycle)
    characterisation_summary = helpers.select_summary_based_on_rate(cell_life)
    life_summary_frames.append(characterisation_summary)
    
    #cell_rate = add_normalized_capacity(cell_rate, norm_cycles)
    rate_summary_frames.append(cell_rate.dataset.dfsummary)
    
    # print("OK")

life_summary_cdf = pd.concat(life_summary_frames, keys=keys, axis=1)
rate_summary_cdf = pd.concat(rate_summary_frames, keys=keys, axis=1)

## Life

In [ ]:
char_curves_overlay = plotutils.concatenated_summary_curve_factory(
    life_summary_cdf, "Normalized_Cycle_Index", 
    "Charge_Capacity(mAh/g)",
    title="Life curves",
)
char_curves_overlay

In [ ]:
life_summary_cdf.to_csv(
    "data/processed/{{cookiecutter.session_id}}_life_cap_vs_norm.csv", 
    sep=";"
)

## Rate

In [ ]:
rate_curves_overlay = plotutils.concatenated_summary_curve_factory(
    rate_summary_cdf, "Cycle_Index", 
    "Normalized_Charge_Capacity(mAh/g)",
    title="Rate curves"
)
rate_curves_overlay

In [ ]:
rate_summary_cdf.to_csv(
    "data/processed/{{cookiecutter.session_id}}_norm_rate_cap_vs_cycle.csv", 
    sep=";"
)

## Notes

- processing raw data [link](01_{{cookiecutter.notebook_name}}_processing.ipynb)